In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import gmaps
import difflib
from config import gkey
gmaps.configure(api_key=gkey)


In [19]:
data = pd.read_csv("liquor_licenses_Denver.csv")
data.head()

,BFN,BUS_PROF_NAME,FULL_ADDRESS,LICENSES,LIC_STATUS,ISSUE_DATE,END_DATE,ADDRESS_ID,ADDRESS_LINE1,ADDRESS_LINE2,CITY,STATE,ZIP,COUNCIL_DIST,POLICE_DIST,CENSUS_TRACT,X_COORD,Y_COORD
0,2013-BFN-1065524,SARTO'S,2900 W 25TH AVE,LIQUOR - HOTEL AND RESTAURANT,LICENSE ISSUED - ACTIVE,2018-08-06 08:34:30,2019-08-21 00:00:00,163150.0,2900 W 25th Ave,None,Denver,CO,None,1,1,00600,3133901.0,1699671.0
1,2006-BFN-1021798,MAD GREENS,1200 ACOMA ST #A-101,LIQUOR - HOTEL AND RESTAURANT,LICENSE ISSUED - ACTIVE,2018-08-23 15:58:52.000001,2019-09-18 00:00:00,None,None,None,None,None,None,10,6,02000,3143825.0,1693407.0
2,2014-BFN-1074718,UNION LODGE NO. 1,1543 CHAMPA STSUITE 101,LIQUOR - TAVERN,LICENSE ISSUED - ACTIVE,2018-11-19 09:43:55,2019-11-30 00:00:00,381438.0,1543 Champa St Ste 101,None,Denver,CO,None,9,6,01702,3142112.0,1697105.0
3,2013-BFN-1066026,TARGET STORE T-2820,7777 E HAMPDEN AVE,LIQUOR - FERMENTED MALT BEVERAGE,LICENSE ISSUED - ACTIVE,2018-06-22 09:07:42,2019-06-26 00:00:00,223542.0,7777 E Hampden Ave,None,Denver,CO,None,4,3,06811,3168865.0,1664167.0
4,2011-BFN-1051932,MARG'S TACO BISTRO,1519 WYNKOOP ST,LIQUOR - HOTEL AND RESTAURANT,LICENSE ISSUED - ACTIVE,2018-06-11 09:10:33.000001,2019-05-01 00:00:00,378005.0,1519 Wynkoop St,None,Denver,CO,None,9,6,01701,3140074.0,1698952.0


In [20]:
# ALl of the below code works (this was the initial code just for Safeway)
# fermented_beer_license = data.loc[data['LICENSES']=='LIQUOR - FERMENTED MALT BEVERAGE']
# Safeway = []
# for bus_name in fermented_beer_license['BUS_PROF_NAME']:
#     Safeway.append(bus_name.find("SAFEWAY"))

# fermented_beer_license['IsSafeway'] = Safeway
# Safeways_I_care_about = fermented_beer_license['IsSafeway'] >= 0
# FinalGroceryStoreList = fermented_beer_license.loc[Safeways_I_care_about]
# FinalGroceryStoreList.head()

In [21]:
# fermented_beer_license = data.loc[data['LICENSES']=='LIQUOR - FERMENTED MALT BEVERAGE']
# grocery = []
# for bus_name in fermented_beer_license['BUS_PROF_NAME']:
#     if bus_name.find("SAFEWAY")>= 0:
#         grocery.append(bus_name.find("SAFEWAY"))
#     elif bus_name.find("KING")>= 0:
#         grocery.append(bus_name.find("KING"))
#     else:
#         grocery.append(int(-1))

# fermented_beer_license['IsGrocery'] = grocery
# grocerys_I_care_about = fermented_beer_license['IsGrocery'] >= 0
# FinalGroceryStoreList = fermented_beer_license.loc[grocerys_I_care_about]


In [22]:
#This cell will request from the maps API to get the 
active_liq_lic = data['FULL_ADDRESS'].head()
lat = []
long = []
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

for city in active_liq_lic:
    params = {"address": city, "key": gkey}
    response = requests.get(base_url, params=params)
    response_json = response.json()
    lat.append(response_json["results"][0]["geometry"]["location"]["lat"])
    long.append(response_json["results"][0]["geometry"]["location"]["lng"])
    
print(lat)
print(long)


[39.7534095, 39.7362402, 39.746276, 39.6547562, 39.7514459]
[-105.0238487, -104.9891889, -104.9946377, -104.9006037, -105.0016923]


In [23]:
active_liq_lic


0             2900 W 25TH AVE
1       1200 ACOMA ST #A-101 
2    1543 CHAMPA STSUITE 101 
3          7777 E HAMPDEN AVE
4             1519 WYNKOOP ST
Name: FULL_ADDRESS, dtype: object

In [24]:
newdf = pd.DataFrame({'Address':active_liq_lic,'Latitude':lat,'Longitude':long})
newdf.head()


,Address,Latitude,Longitude
0,2900 W 25TH AVE,39.753409,-105.023849
1,1200 ACOMA ST #A-101,39.736240,-104.989189
2,1543 CHAMPA STSUITE 101,39.746276,-104.994638
3,7777 E HAMPDEN AVE,39.654756,-104.900604
4,1519 WYNKOOP ST,39.751446,-105.001692


In [25]:
fig = gmaps.figure(center=(39.7392,-104.9903), zoom_level=12)
active_license_locations = newdf[['Latitude','Longitude']]
heat_layer = gmaps.heatmap_layer(active_license_locations)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# gmaps.symbol_layer(locations, hover_text='', fill_color=None, fill_opacity=1.0, stroke_color=None, stroke_opacity=1.0, scale=3, info_box_content=None, display_info_box=None)